## Pixelated Image Detection Model

### Extracting Dataset from ZIP File

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('dataset1.zip', 'r')
zip_ref.extractall()
zip_ref.close

<bound method ZipFile.close of <zipfile.ZipFile filename='dataset1.zip' mode='r'>>

### Importing Required Libraries

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

### Defining Dataset Paths

In [ ]:
# Define the paths to the dataset
train_dataset_path = './dataset1/train/'
test_dataset_path = './dataset1/test/'

### Creating Data Generators

In [ ]:
# Create a data generator for loading and augmenting the data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize the pixel values
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize the pixel values
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

### Creating Training and Validation Generators

In [ ]:
# Create generators for training and validation sets
train_generator = train_datagen.flow_from_directory(
    train_dataset_path,
    target_size=(256, 256),  # Change target size to 256x256
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    test_dataset_path,
    target_size=(256, 256),  # Change target size to 256x256
    batch_size=32,
    class_mode='binary'
)

### Building the Model

In [ ]:
# Build the CNN model
model = Sequential([
    Conv2D(8, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

### Compiling the Model

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

### Training the Model

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50
)

### Evaluating the Model

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test accuracy: {test_acc}')

### Saving the Model

In [ ]:
# Save the model
model.save('pixelation_detector_model.h5')

### Getting the summary of the model

In [ ]:
model.summary()

### Loading the Trained Model

In [ ]:
# Load the trained model
model = load_model('pixelation_detector_model.h5')

### Defining Sample Image Path

In [ ]:
# Define the path to the sample image
sample_image_path = 'dataset/test/Non_Pixelated/167.jpg'

### Loading and Preprocessing the Sample Image

In [ ]:
# Load and preprocess the sample image
img = image.load_img(sample_image_path, target_size=(256, 256))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

### Making a Prediction

In [ ]:
# Make a prediction
prediction = model.predict(img_array)
predicted_class = 'Pixelated' if prediction[0][0] > 0.5 else 'Not Pixelated'

print(f'The model predicts the image is: {predicted_class}')